In [1]:
pip uninstall tensorflow

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1


In [2]:
pip uninstall numpy

Uninstalling numpy-1.19.5:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.19.5


In [3]:
pip install numpy==1.19

     |████████████████████████████████| 14.6MB 5.4MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [4]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 512kB 58.8MB/s 
     |████████████████████████████████| 3.8MB 53.7MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=83705bede846351cf05ad39b4140795686742807356c86685a79f4be6cd2d542
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [1]:
from __future__ import print_function, division
from __future__ import absolute_import

import argparse
import sys
import numpy as np
import cv2
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import json
import collections
import matplotlib.pyplot as plt

from tensorflow import keras
# from keras.datasets import mnist
# from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
# from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
# from keras.layers.advanced_activations import LeakyReLU
# from keras.layers.convolutional import UpSampling2D, Conv2D
# from keras.models import Sequential, Model
# from keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
keras.__version__

'2.2.4-tf'

In [17]:
np.__version__

'1.19.5'

In [2]:
def crop_image(img):
    image = np.array(img)
    blur = cv2.GaussianBlur(image, ksize=(3,3), sigmaX=0)
    ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(blur, 10, 250)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total = 0
    contours_xy = np.array(contours)
    x_min, x_max = 0,0
    value = list()
    for i in range(len(contours_xy)):
      for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][0])
        x_min = min(value)
        x_max = max(value)
 
    y_min, y_max = 0,0
    value = list()
    for i in range(len(contours_xy)):
      for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][1])
        y_min = min(value)
        y_max = max(value)

    x = x_min
    y = y_min
    w = x_max-x_min
    h = y_max-y_min

    return x, y, w, h

def process_image(img, x, y, w, h, canvas_size):
    new_width = 50
    new_height = int(50 * h / w)
    if new_height > 60:
        new_height = 55
        new_width = int(55 * w / h)
    img = img.crop((x-1, y-1, x+w+1, y+h+1)).resize((new_width,new_height))
    new_left = int((64 - img.width) / 2)
    new_top = int((64 - img.height) / 2)
    result = Image.new("L", (canvas_size, canvas_size), color=255)
    result.paste(img, (new_left, new_top))

    return result

def draw_single_char(ch, font, canvas_size, x_offset, y_offset):
    img = Image.new("L", (150, 150), color=255)
    draw = ImageDraw.Draw(img)
    # draw.text((x_offset, y_offset), ch, fill=(0), font=font, stroke_width=2, stroke_fill="black")
    draw.text((x_offset, y_offset), ch, fill=(0), font=font)
    x, y, w, h = crop_image(img)
    img = process_image(img, x, y, w, h, canvas_size)
    return img


def font2img(ttf, charset, char_size, canvas_size, x_offset, y_offset, folder):
    global X_train, y_train
    font = ImageFont.truetype(ttf, size=char_size)
    count = 0
    for c in charset:
        e = draw_single_char(c, font, canvas_size, x_offset, y_offset)
        if e:
            e.save(folder+c+'.png')
            # X_train.append(np.array(e))
            # y_train.append(count)
            count += 1

In [4]:
path = './gdrive/MyDrive/hangul/ttf/'
fonts = os.listdir(path)
charset = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
char_size, canvas_size = 100, 64
x_offset, y_offset = 10, 10
label = 1
X_train = []
y_train = []

srcfont = fonts[0]
font2img(path + srcfont, charset, char_size, canvas_size, x_offset, y_offset, 'src/')

dstfont = fonts[1]
font2img(path + dstfont, charset, char_size, canvas_size, x_offset, y_offset, 'dst/')

# for font in fonts:
#     font2img(path + font, charset, char_size, canvas_size, x_offset, y_offset)
#     print('font#%02d 데이터셋 생성 완료' % (label))
#     label += 1

In [4]:
np.array(Image.open('src/A.png')).shape

(64, 64)

In [5]:
# encoding=UTF-8
import numpy as np
import matplotlib as plt
from PIL import Image
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# %%
def data_preprocess(path='',
                    source_font='src',
                    target_font='dst'):
    """
        normalize font images to [0,1] and then create the source-target font pairs
    """
    # load the source fonts and target fonts
    source_path = source_font #path+'/'+source_font
    source_fonts = list()
    target_path = target_font #path + '/' + target_font
    target_fonts = list()
    files = os.listdir(source_path)
    mid1 = np.zeros((64, 64, 1))
    mid2 = np.zeros((64, 64, 1))   

    
    for file in files:
        #load a source font
        img1 = cv2.imread(source_path+'/'+file) #Image.open(source_path+'/'+file)
        np1 = np.array(img1)
        mid1 = np1[:, :, 0]
        mid1 = mid1[:,:,np.newaxis]
        source_fonts.append(mid1)

        #load a target font
        img2 = cv2.imread(source_path+'/'+file) #Image.open(target_path + '/'+file)
        np2 = np.array(img2)
        mid2 = np2[:, :, 0]
        mid2 = mid2[:,:,np.newaxis]
        target_fonts.append(mid2)

    source_fonts = np.array(source_fonts)
    source_fonts = source_fonts.astype(np.float32)
    source_fonts /= 255

    target_fonts = np.array(target_fonts)
    target_fonts = target_fonts.astype(np.float32)
    target_fonts /= 255


    return source_fonts, target_fonts


class Dataset:
    def __init__(self, source_fonts, target_fonts, test_frac=0.4, val_frac=0.1, scale_func=None):
        """
            create the training set, validation set and testing set
        """
        self.data_num = int(len(source_fonts))
        self.val_num = int(self.data_num*(1-test_frac)*val_frac)
        self.train_num = int(self.data_num * (1 - test_frac)) - self.val_num
        self.test_num = self.data_num-self.val_num-self.train_num

        self.train = {}
        self.test = {}
        self.valid = {}


        """self.train['source_font'] = np.rollaxis(
            source_fonts[:self.train_num], axis=3)
        self.valid['source_font'] = np.rollaxis(
            source_fonts[self.train_num: self.train_num+self.val_num], axis=3)
        self.test['source_font'] = np.rollaxis(
            source_fonts[self.train_num + self.val_num:], axis=3)
        self.train['target_font'] = np.rollaxis(
            target_fonts[:self.train_num], axis=3)
        self.valid['target_font'] = np.rollaxis(
            target_fonts[self.train_num: self.train_num+self.val_num], axis=3)
        self.test['target_font'] = np.rollaxis(
            target_fonts[self.train_num + self.val_num:], axis=3)"""

        self.train['source_font'] =source_fonts[:self.train_num]
        self.valid['source_font'] =source_fonts[self.train_num: self.train_num+self.val_num]
        self.test['source_font'] = source_fonts[self.train_num + self.val_num:]

        self.train['target_font'] =target_fonts[:self.train_num]
        self.valid['target_font'] =target_fonts[self.train_num: self.train_num+self.val_num]
        self.test['target_font'] =target_fonts[self.train_num + self.val_num:]

    def shuffle_data( self ):
        idx = np.arange(self.train_num)
        np.random.shuffle(idx)
        self.train['source_font'], self.train['target_font'] = self.train['source_font'][idx], self.train['target_font'][idx]

    def get_batches(self, batch_size):
        """
            generate one batch of data
        """
        batch_num = self.train_num // batch_size

        for ii in range(0, batch_num * batch_size, batch_size):
            source_font = self.train['source_font'][ii:ii+batch_size]
            target_font = self.train['target_font'][ii:ii+batch_size]

            yield source_font, target_font


# %%
#source_fonts, target_fonts = data_preprocess()


# %%

#dataset = Dataset( source_fonts, target_fonts)
#print(dataset.train['source_font'].shape)

# 解决getbatches报错：'KeyValueError target_font'
# %%

In [6]:
  
# %%
import tensorflow as tf
import numpy as np
import matplotlib as plt
from PIL import Image
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# %%
# hyperparameters
image_dim = (64, 64, 1)
learning_rate = 0.005
batch_size = 32
element_num = batch_size*image_dim[0]*image_dim[1]*image_dim[2]
epochs = 30
beta1 = 0.9
lambda_p = 1
lambda_1 = 10
lambda_2 = 1
lambda_3 = 1
lambda_4 = 30
print_every = 1


# %%

def leaky_relu(x):
    alpha = -0.1
    return tf.where( x >= 0.0, x, alpha*x )

def selu(x):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale*tf.where(x >= 0.0, x, alpha*tf.nn.elu(x))


# %%
def generator(source_font, reuse=False, training=True):

    with tf.variable_scope('generator', reuse=reuse):
        # input tensor size is 64*64*1
        input_layer = tf.reshape(source_font, (-1, 64, 64, 1))

        # 64*64*1 -> 64*64*64
        conv1 = tf.layers.conv2d(
            inputs=input_layer, filters=64, kernel_size=3, padding='same')
        conv1 = tf.layers.batch_normalization(conv1, training=True)
        conv1 = leaky_relu(conv1)

        # 128*128*64 -> 64*64*128
        conv2 = tf.layers.conv2d(
            inputs=conv1, filters=64, kernel_size=3, strides=2, padding='same')
        conv2 = tf.layers.batch_normalization(conv2, training=True)
        conv2 = leaky_relu(conv2)
        conv2 = tf.layers.conv2d(
            inputs=conv2, filters=128, kernel_size=3, padding='same')
        conv2 = tf.layers.batch_normalization(conv2, training=True)
        conv2 = leaky_relu(conv2)

        # 64*64*128 -> 32*32*256
        conv3 = tf.layers.conv2d(
            inputs=conv2, filters=128, kernel_size=3, strides=2, padding='same')
        conv3 = tf.layers.batch_normalization(conv3, training=True)
        conv3 = leaky_relu(conv3)
        conv3 = tf.layers.conv2d(
            inputs=conv3, filters=256, kernel_size=3, padding='same')
        conv3 = tf.layers.batch_normalization(conv3, training=True)
        conv3 = leaky_relu(conv3)

        # 32*32*256 -> 16*16*512
        conv4 = tf.layers.conv2d(
            inputs=conv3, filters=256, kernel_size=3, strides=2, padding='same')
        conv4 = tf.layers.batch_normalization(conv4, training=True)
        conv4 = leaky_relu(conv4)
        conv4 = tf.layers.conv2d(
            inputs=conv4, filters=512, kernel_size=3, padding='same')
        conv4 = tf.layers.batch_normalization(conv4, training=True)
        conv4 = leaky_relu(conv4)

        # 16*16*512 -> 8*8*512
        conv5 = tf.layers.conv2d(
            inputs=conv4, filters=512, kernel_size=3, strides=2, padding='same')
        conv5 = tf.layers.batch_normalization(conv5, training=True)
        conv5 = leaky_relu(conv5)

        # 8*8*512 -> 16*16*512
        # 16*16*512 + 16*16*512 -> 16*16*1024
        # 16*16*1024 -> 16*16*512
        # 16*16*512 -> 16*16*256
        up6 = tf.layers.conv2d_transpose(
            inputs=conv5, filters=512, kernel_size=3, strides=2, padding='same')
        up6 = tf.layers.batch_normalization(up6, training=True)
        up6 = leaky_relu(up6)
        up6 = tf.concat([conv4, up6], axis=3)
        conv6 = tf.layers.conv2d(
            inputs=up6, filters=512, kernel_size=3, padding='same')
        conv6 = tf.layers.batch_normalization(conv6, training=True)
        conv6 = leaky_relu(conv6)
        conv6 = tf.layers.conv2d(
            inputs=conv6, filters=256, kernel_size=3, padding='same')
        conv6 = tf.layers.batch_normalization(conv6, training=True)
        conv6 = leaky_relu(conv6)

        # 16*16*256 -> 32*32*256
        # 32*32*256 + 32*32*256 -> 32*32*512
        # 32*32*512 -> 32*32*256
        # 32*32*256 -> 32*32*128
        up7 = tf.layers.conv2d_transpose(
            inputs=conv6, filters=256, kernel_size=3, strides=2, padding='same')
        up7 = tf.layers.batch_normalization(up7, training=True)
        up7 = leaky_relu(up7)
        up7 = tf.concat([conv3, up7], axis=3)
        conv7 = tf.layers.conv2d(
            inputs=up7, filters=256, kernel_size=3, padding='same')
        conv7 = tf.layers.batch_normalization(conv7, training=True)
        conv7 = leaky_relu(conv7)
        conv7 = tf.layers.conv2d(
            inputs=conv7, filters=128, kernel_size=3, padding='same')
        conv7 = tf.layers.batch_normalization(conv7, training=True)
        conv7 = leaky_relu(conv7)

        # 32*32*128 -> 64*64*128
        # 64*64*128 + 64*64*128 -> 64*64*256
        # 64*64*256 -> 64*64*128 -> 64*64*64
        up8 = tf.layers.conv2d_transpose(
            inputs=conv7, filters=128, kernel_size=3, strides=2, padding='same')
        up8 = tf.layers.batch_normalization(up8, training=True)
        up8 = leaky_relu(up8)
        up8 = tf.concat([conv2, up8], axis=3)
        conv8 = tf.layers.conv2d(
            inputs=up8, filters=128, kernel_size=3, padding='same')
        conv8 = tf.layers.batch_normalization(conv8, training=True)
        conv8 = leaky_relu(conv8)
        conv8 = tf.layers.conv2d(
            inputs=conv8, filters=64, kernel_size=3, padding='same')
        conv8 = tf.layers.batch_normalization(conv8, training=True)
        conv8 = leaky_relu(conv8)

        # 64*64*64 -> 128*128*64
        # 128*128*64 + 128*128*64 -> 128*128*128
        # 128*128*128 -> 128*128*64 -> 128*128*1
        up9 = tf.layers.conv2d_transpose(
            inputs=conv8, filters=64, kernel_size=3, strides=2, padding='same')
        up9 = tf.layers.batch_normalization(up9, training=True)
        up9 = leaky_relu(up9)
        up9 = tf.concat([conv1, up9], axis=3)
        conv9 = tf.layers.conv2d(
            inputs=up9, filters=64, kernel_size=3, padding='same')
        conv9 = tf.layers.batch_normalization(conv9, training=True)
        conv9 = leaky_relu(conv9)
        conv9 = tf.layers.conv2d(
            inputs=conv9, filters=1, kernel_size=3, padding='same')
        conv9 = tf.layers.batch_normalization(conv9, training=True)
        conv9 = tf.nn.sigmoid(conv9)

        return conv9


# %%
def discriminator(x, reuse=False):
    """
        x: a real image of target fonts or a fake image generated by generator use source fonts
    """
    with tf.variable_scope('discriminator', reuse=reuse):
        w_init = tf.contrib.layers.xavier_initializer()

        conv1 = tf.layers.conv2d(
            inputs=x, filters=256, kernel_size=3, strides=2, padding='same')
        conv1 = tf.layers.batch_normalization(conv1, training=True)
        conv1 = leaky_relu(conv1)

        conv2 = tf.layers.conv2d(
            inputs=conv1, filters=512, kernel_size=3, strides=2, padding='same')
        conv2 = tf.layers.batch_normalization(conv2, training=True)
        conv2 = leaky_relu(conv2)

        conv3 = tf.layers.conv2d(
            inputs=conv2, filters=1024, kernel_size=3, strides=2, padding='same')
        conv3 = tf.layers.batch_normalization(conv3, training=True)
        conv3 = leaky_relu(conv3)

        conv4 = tf.layers.conv2d(
            inputs=conv3, filters=2048, kernel_size=3, strides=2, padding='same')
        conv4 = tf.layers.batch_normalization(conv4, training=True)
        conv4 = leaky_relu(conv4)

        flat = tf.reshape(conv4, (-1, 8*8*2048))
        logits = tf.layers.dense(flat, 1)
        out = tf.nn.sigmoid(logits)

        return out


# %%
class TransNN:
    def __init__(self):
        tf.reset_default_graph()

    '''def input_setup(self, image_dim):
        
        self.target_fonts = tf.placeholder(
            tf.float32, (None,  128, 128, 1), name='target_font')
        self.source_fonts = tf.placeholder(
            tf.float32, (None,  128, 128, 1), name='source_font')'''

    def model_setup(self):
        '''
        This function sets up the model to train
        '''
        self.target_fonts = tf.placeholder(
            tf.float32, (None,  64, 64, 1), name='target_fonts')
        self.source_fonts = tf.placeholder(
            tf.float32, (None,  64, 64, 1), name='source_fonts')

        self.generated_fonts = generator(self.source_fonts)
        self.real_score = discriminator(self.target_fonts)
        self.fake_score = discriminator(self.generated_fonts, reuse=True)

    def model_loss(self):
        #pix2pix cross_entropy loss
        p2p_cross_entropy = tf.reduce_sum(- lambda_p * np.multiply(self.target_fonts, tf.log(self.generated_fonts)
                                                         ) - np.multiply(1-self.target_fonts, tf.log(1-self.generated_fonts)))
        diffrence = tf.subtract(self.target_fonts, self.generated_fonts)
        #l1-norm
        l1_norm = tf.reduce_sum( tf.abs(diffrence))
        #l2-norm
        l2_norm = tf.reduce_sum( np.multiply( diffrence, diffrence))

        v1_loss = lambda_1 * p2p_cross_entropy + lambda_4 * l1_norm 
        v1_loss /= element_num
        #v1_loss = v1_loss / target_font.shape[0]

        # v2_loss evaluates the discriminator performence of seperating generated fonts from target fonts
        v2_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.real_score, labels=tf.ones_like(
            self.real_score))) + tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.fake_score, labels=tf.zeros_like(self.fake_score)))

        # v3_loss evaluate the generator performence of generating real fonts
        v3_loss_log = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=self.fake_score, labels=tf.ones_like(self.fake_score)))
        # in case of gradient varnishment, we use v3_loss without log
        v3_loss = tf.reduce_sum(self.fake_score)


        #self.generator_loss =  ( lambda_1 * v1_loss + lambda_2 * v3_loss_log ) / batch_size
        #self.generator_loss =  ( lambda_1 * v1_loss + lambda_2 *( batch_size -  v3_loss ) ) / batch_size
        self.generator_loss =  v1_loss  / batch_size 

        self.discriminator_loss = ( lambda_3 * v2_loss ) / batch_size

        self.model_vars = tf.trainable_variables()

        d_vars = [var for var in self.model_vars if var.name.startswith(
            'discriminator')]
        g_vars = [
            var for var in self.model_vars if var.name.startswith('generator')]

        self.d_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(
            self.discriminator_loss, var_list=d_vars)
        self.g_train_opt = tf.train.AdamOptimizer(
            learning_rate, beta1=beta1).minimize(self.generator_loss, var_list=g_vars)

    def train(self):
        self.model_setup()

        self.model_loss()

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        with tf.Session(config=config) as sess:
            sess.run(init)

            # Training
            for epoch_no in range(epochs):
                dataset.shuffle_data()

                batch_no = 0
                for source_font, target_font in dataset.get_batches(batch_size):
                    batch_no += 1
                    print("Batch No: {}/{} ".format(batch_no,
                                                    dataset.train_num // batch_size))

                    # Use generator to generate fonts from source fonts
                    sess.run(self.generated_fonts, feed_dict={
                             self.source_fonts: source_font, self.target_fonts: target_font})

                    # Optimize the discriminator
                    sess.run(self.d_train_opt, feed_dict={self.target_fonts: target_font, self.source_fonts: source_font})

                    # Optimize the generator
                    sess.run(self.g_train_opt, feed_dict={
                             self.target_fonts: target_font, self.source_fonts: source_font})

                    if epoch_no % print_every == 0:
                        train_loss_d = self.discriminator_loss.eval(
                            {self.target_fonts: target_font, self.source_fonts: source_font})
                        train_loss_g = self.generator_loss.eval(
                            {self.source_fonts: source_font, self.target_fonts: target_font})

                        print("Epoch {}/{}...".format(epoch_no+1, epochs),
                              "Discriminator Loss: {:.8f}...".format(
                                  train_loss_d),
                              "Generator Loss: {:.8f}".format(train_loss_g))
                        # Save losses to view after training
                        #losses.append((train_loss_d, train_loss_g))

            saver.save(sess, './checkpoints/generator')

    def test(self):
        print("Testing the results")

        # self.model_setup()

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        with tf.Session(config=config) as sess:
            sess.run(init)

            saver = tf.train.import_meta_graph('./checkpoints/generator.meta')
            saver.restore(sess, tf.train.latest_checkpoint('./checkpoints/'))
            graph = tf.get_default_graph()
            image_no = 0

            for num in range(0, dataset.test_num, batch_size):
                generated_fonts = sess.run(self.generated_fonts, feed_dict={
                                           self.source_fonts: dataset.test['source_font'][num:num+batch_size], self.target_fonts: dataset.test['target_font'][num:num+batch_size]})
                source_fonts = dataset.test['target_font'][num:num+batch_size]

                for batch_no in range(min(batch_size, generated_fonts.shape[0])):
                    generated_font = generated_fonts[batch_no]
                    target_font = source_fonts[batch_no]

                    mid = np.append(generated_font, generated_font, axis=2)
                    mid = np.append(generated_font, mid, axis=2)
                    mid = mid*255
                    mid = mid.astype('uint8')
                    img = Image.fromarray(mid)
                    img.save(fp='./generated_fonts/generated_' +
                             str(image_no) + '.jpg')

                    mid[:, :, :] = target_font
                    mid = mid*255
                    mid = mid.astype('uint8')
                    img = Image.fromarray(mid)
                    img.save(fp='./generated_fonts/target_' +
                             str(image_no)+'.jpg')

                    image_no += 1


# %%
# writer=tf.summary.FileWriter(r"/Users/hangyizhe/GitHub/Chinese_Font_Transfer/models",tf.get_default_graph())
# writer.close()
source_fonts, target_fonts = data_preprocess()
dataset = Dataset(source_fonts, target_fonts, test_frac=0.1, val_frac=0)

# %%
net = TransNN()
net.train()
print("Training Finished!")

net.test()

print("Test Finished!")


# %%

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.


NotImplementedError: ignored